# COVID-19 Dashboard

Welcome to my COVID-19 dashboard, which illustrates the extensive and lasting impact of the recent pandemic. This dashboard aims to visually communicate the effects on public health, heightened societal awareness of health issues, and the enduring socio-economic challenges that continue to ripple through our communities. Drawing on the latest data from the Office for National Statistics (ONS) as of March 2023, it's evident that COVID-19 remains a significant health concern. In England alone, approximately 1 in 40 people are still testing positive, and the virus is responsible for 4.5% of deaths within the week. This ongoing situation is mirrored in public perception, with a notable portion of the population, around "1 in 5 adults" (Office for National Statistics, 2023), still viewing the coronavirus as a present and active threat. This is further evidenced by the same proportion of individuals continuing to use face coverings outside their homes, reflecting a collective effort to mitigate the spread of the virus.

In this dashboard, you will find two interactive graphs. These graphs are designed to display some critical data points for comparison and also offer a dynamic and engaging way to understand the evolution and current state of the pandemic. By interacting with these graphs, you can explore various areas of COVID-19's impact, from hospital admissions to public health measures.

The intention behind this dashboard is to provide an accessible view of the pandemic’s trajectory and its effects. It aims to deepen your understanding of how COVID-19 has shaped our world and to highlight the importance of continued awareness and adaptation in our responses.

In [9]:
# import necessary modules and libraries
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 125

In [10]:
# function to grab list of data from JSON

def extract_data(file_name):
    with open(f"./JSONs/{file_name}.json", "rt") as FILE:
        jsondata=json.load(FILE)

    return jsondata['data']

# defining variables containing the extracted JSON data for graph one and two
# also define their metric variables here, just to reduce code repetition

one_data = extract_data('tests_cases')
one_metrics = ['tests', 'cases']
two_data = extract_data('cases_admissions_occupancies')
two_metrics = ['cases', 'admissions', 'occupancies']

In [11]:
# below are functions relating to the management of graph data

def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def wrangle_data(rawdata, metrics):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe. """

    # use a list comprehension to generate a list of dates in the json data variable
    dates = [item['date'] for item in rawdata]
    dates.sort()

    # set start and end dates to a pandas format
    startdate = parse_date(dates[0])
    enddate = parse_date(dates[-1])

    # initialise empty dataframe
    index = pd.date_range(startdate, enddate, freq='D')
    df = pd.DataFrame(index=index, columns=metrics)

    # check for each date and column combination in JSON, and fill if empty
    for entry in rawdata:
        date = parse_date(entry['date'])
        for column in metrics:
            if pd.isna(df.loc[date, column]): 
                value = entry[column] if entry[column]!= None else 0
                df.loc[date, column] = value

    # set remaining cells to 0 to get rid of n/a values
    df.fillna(0, inplace=True)

    # return filled-in dataframe, ready to be plotted
    return df

# create dataframes for graph one and two using locally-stored JSON data defined earlier
df_one = wrangle_data(one_data, one_metrics)
df_two = wrangle_data(two_data, two_metrics)

In [12]:
# below are functions related to fetching API data and refreshing graphs

def access_api(file_name, filters, structure):
    """ Accesses the PHE API. Return data as a like-for-like replacement for the
    "canned" data loaded from the JSON file. """

    api = Cov19API(filters=filters, structure=structure)
    api_data = api.get_json()

    with open(f"./JSONs/{file_name}.json", "wt") as FILE:
        json.dump(api_data, FILE)

    return api_data['data']


def refresh_graph(filter_obj):
    """ We change the value of the widget in order to force a redraw of the graph. """
    current = filter_obj.value
    if len(current) == 1:
        # get value of current index as a string
        current = filter_obj.value[0]
        # get the index within the filter's options by using index() method
        current_index = filter_obj.options.index(current)
        # set a temp index different to current value's index
        temp_index = current_index - 1
        # set filter's value to the temp index's string value represented as a tuple (e.g., ('tests',))
        filter_one.value = tuple([filter_obj.options[temp_index]])
        # with the same logic, set it back to the original value - this should refresh the graph
        filter_one.value = tuple([filter_obj.options[current_index]])
    else:
        # for cases where len(current) != 1, as I cannot use index() on a multi-valued tuple
        # just get one of the values in the tuple - all that matters is that it differs from the current tuple
        temp_value = current[0]
        # temporarily set filter's value to this temp value
        filter_obj.value = tuple([temp_value])
        # reset value to refresh graph
        filter_obj.value = current
        
def api_button_callback(button_obj, file_name, filters, structure, graphno, filter_obj):
    """ Button callback function """
    try:
        api_data = access_api(file_name, filters, structure)
        
        global df_one
        global df_two
    
        if graphno == 1:
            df_one = wrangle_data(api_data, one_metrics)
        else:
            df_two = wrangle_data(api_data, two_metrics)
    
        refresh_graph(filter_obj)
        button_obj.icon = "check"
        button_obj.disabled = True
        button_obj.button_style = 'success'
    # exception handling in case API call went wrong
    except Exception:
        button_obj.icon = "times"
        button_obj.button_style = "danger"
        # change text on button to relay error message to user
        button_obj.description = "API Error"
        button_obj.disabled = True

## Tests, Cases

The graph below depicts the number of new COVID-19 tests taken, alongside the new COVID-19 cases confirmed from July 2021 to 2023. We observe a clear correlation: when testing ramps up, case numbers tend to spike, likely aligning with the infection waves. Through the latter half of 2021 and into early 2022, these spikes are quite pronounced, highlighting strong periods of testing and case discovery.

Moving into 2022, there’s a noticeable decrease in both testing and cases, which could be due to the rollout of vaccinations and the enforcement of effective public health policies. Despite this decline, the graph does point out occasional spikes in cases post-2022. These spikes could perhaps be partly due to the reduced testing frequency, suggesting that when fewer tests are conducted, cases may go undetected, leading to a potential rise in unreported infections and allowing spread.

Overall, the trend appears to show a reduction in the pandemic's impact over time. This downward trend is promising, suggesting that measures taken have been effective. Yet, the data also highlights the importance of continuous testing, as shown in the post-2022 spikes. Regular testing is key for catching new cases early and helping to prevent another wave of infections.

Click the 'Refresh Data' button below to refresh the graph to the latest API data.

In [13]:
# function to plot graph one based on the filters selected
def plot_graph_one(filters, scale=None):
    """ Plot graph one"""
    logcheck = (scale == 'log scale')
    
    if len(filters) > 0:
        df_one[list(filters)].plot(logy=logcheck)
        plt.show()
    else:
        print("\nClick to filter, or Shift/Ctrl Click to select multiple")
        

# widget to allow multiple selection using graph one metrics defined earlier
filter_one = wdg.SelectMultiple(
    options = one_metrics,
    value = one_metrics,
    rows = 2,
    description = 'Filter:',
    disabled = False
)

# widget to set scale of graph one
scale_one = wdg.RadioButtons(
    options = ['default scale', 'log scale'],
    disabled = False
)

control_container_one = wdg.HBox([filter_one, scale_one])

    
# connects the plotting function and the widget    
graph_one = wdg.interactive_output(plot_graph_one, {'filters': filter_one, 'scale': scale_one})

# display graph
display(control_container_one, graph_one)

Output()

In [14]:
filters_one = ['areaType=nation', 'areaName=England']
    
structure_one = {
    'date': 'date',
    'tests': 'newTestsByPublishDate',
    'cases': 'newCasesByPublishDate'
}

button_one = wdg.Button(
    description = 'Refresh Data',
    disabled = False,
    button_style = 'warning',
    tooltip = "Click button to refresh data",
    icon = 'download'
)

# using a lambda function here to allow me to pass the callback function with
# multiple arguments without evoking the function prematurely
# necessary if I want to reuse the button code for both graphs to avoid repetition
button_one.on_click(lambda x: api_button_callback(button_one, 'tests_cases', filters_one, structure_one , 1, filter_one))
# here, graphno is set to '1' to indicate the global variable 'df_one' should be updated
display(button_one)

Button(button_style='warning', description='Refresh Data', icon='download', style=ButtonStyle(), tooltip='Clic…

## Cases, Hospital Admissions, Bed Occupancy

The graph below provides a side-by-side comparison of hospital admissions and bed occupancy for COVID-19 from mid-2020 through mid-2023. The data reveals that admissions and bed occupancy move together, suggesting a direct link—more admissions typically mean more beds filled, which makes sense. The early part of the graph shows steep peaks, which reflect the intense pressure on hospitals during the early and most severe phases of the pandemic. Moving forward in time, a gradual decrease in both admissions and occupancy is apparent, likely due to the rollout of vaccinations and improvements in treatments.

Despite this overall trend of decline, there are still noticeable, though smaller, spikes in hospital activity through to 2023. These indicate that hospitals occasionally face the challenge of managing a rising number of COVID-19 patients. The reduced height of these later spikes could suggest that the pressure on hospital resources is not as intense as it was earlier in the pandemic.

Overall, this graph highlights the need for hospitals to maintain readiness for potential future spikes and highlights the importance of ongoing public health measures to control the impact of the virus on healthcare systems.

Click the 'Refresh Data' button below to refresh the graph to the latest API data.

In [15]:
# function to plot graph two based on the filters selected
def plot_graph_two(filters, scale=None):
    """ Plot graph two"""
    logcheck = (scale == 'log scale')
    
    if len(filters) > 0:
        df_two[list(filters)].plot(logy=logcheck)
        plt.show()
    else:
        print("\nClick to filter, or Shift/Ctrl Click to select multiple")
        

# widget to allow multiple selection using graph two metrics defined earlier
filter_two = wdg.SelectMultiple(
    options = two_metrics,
    value = ['admissions', 'occupancies'],
    rows = 3,
    description = 'Filter:',
    disabled = False
)

# widget to set scale of graph two
scale_two = wdg.RadioButtons(
    options = ['default scale', 'log scale'],
    disabled = False
)

control_container_two = wdg.HBox([filter_two, scale_two])

    
# connects the plotting function and the widget    
graph_two = wdg.interactive_output(plot_graph_two, {'filters': filter_two, 'scale': scale_two})

# display graph
display(control_container_two, graph_two)

Output()

In [16]:
filters_two = ['areaType=nation', 'areaName=England']

structure_two = {
    'date': 'date',
    'cases': 'newCasesByPublishDate',
    'admissions': 'newAdmissions',
    'occupancies': 'covidOccupiedMVBeds'
}

button_two = wdg.Button(
    description = 'Refresh Data',
    disabled = False,
    button_style = 'warning',
    tooltip = "Click button to refresh data",
    icon = 'download'
)

# using a lambda function here to allow me to pass the callback function with
# multiple arguments without evoking the function prematurely
button_two.on_click(lambda x: api_button_callback(button_two, 'cases_admissions_occupancies', filters_two, structure_two , 2, filter_two))
# here, graphno is set to '2' to indicate the global variable 'df_two' should be updated
display(button_two)

Button(button_style='warning', description='Refresh Data', icon='download', style=ButtonStyle(), tooltip='Clic…

### References

Office for National Statistics. (2022, March 30). Coronavirus (COVID-19) latest insights - Office for National Statistics. Www.ons.gov.uk. https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19/latestinsights

### Author and Copyright Notice

Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england) and on the [DIY Covid Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash)

Copyright (C) Carlo Lopez, 2023. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).